In [23]:
from pathlib import Path
import os

In [3]:
base_path = Path("../data/full_door_window")
# Директории
train_img_dir = base_path / "train" / "images"
train_lbl_dir = base_path / "train" / "labels"

valid_img_dir = base_path / "valid" / "images"
valid_lbl_dir = base_path / "valid" / "labels"

test_img_dir = base_path / "test" / "images"
test_lbl_dir = base_path / "test" / "labels"

yaml_path = base_path / "data.yaml" 

In [17]:
def remove_class_from_labels(label_dir, class_ids_to_remove):
    for filename in os.listdir(label_dir):
        if filename.endswith('.txt'):
            path = os.path.join(label_dir, filename)
            with open(path, 'r') as file:
                lines = file.readlines()
            # Удалим строки, начинающиеся с любого class_id из списка
            new_lines = [
                line for line in lines 
                if not any(line.startswith(f"{class_id} ") for class_id in class_ids_to_remove)
            ]
            with open(path, 'w') as file:
                file.writelines(new_lines)


In [20]:
# Применим для train и valid
remove_class_from_labels("../data/full_door_window/train/labels", [0, 1, 2, 3, 4, 5, 7, 8])
remove_class_from_labels("../data/full_door_window/valid/labels", [0, 1, 2, 3, 4, 5, 7, 8])
remove_class_from_labels("../data/full_door_window/test/labels", [0, 1, 2, 3, 4, 5, 7, 8])

In [22]:
# Папки с аннотациями
label_dirs = [
    Path("../data/full_door_window/test/labels"),
    Path("../data/full_door_window/valid/labels"),
    Path("../data/full_door_window/train/labels")
]

# Перенумерация классов: старая -> новая
id_map = {6: 0, 9: 1}

for label_dir in label_dirs:
    print(f"\n🗂 Обработка папки: {label_dir}")

    for label_file in label_dir.glob("*.txt"):
        changed = False
        new_lines = []

        with open(label_file, "r") as f:
            lines = f.readlines()

        for line_num, line in enumerate(lines, 1):
            parts = line.strip().split()
            if len(parts) != 5:
                print(f"⚠️  Пропущена строка с ошибкой в {label_file.name}, строка {line_num}: {line.strip()}")
                continue

            try:
                old_id = int(parts[0])
            except ValueError:
                print(f"❌ Невозможно преобразовать class_id в число: {parts[0]}")
                continue

            if old_id in id_map:
                parts[0] = str(id_map[old_id])
                changed = True
                new_lines.append(" ".join(parts))
            else:
                print(f"❌ Неизвестный class_id {old_id} в {label_file.name}, строка {line_num}")
                continue

        if changed:
            with open(label_file, "w") as f:
                f.write("\n".join(new_lines) + "\n")
            print(f"✅ Обновлён: {label_file.name}")
        else:
            print(f"ℹ️ Без изменений: {label_file.name}")




🗂 Обработка папки: ../data/full_door_window/test/labels
✅ Обновлён: 70_png.rf.b5c6c6d8236a78017667049e46ca5415.txt
✅ Обновлён: 76_png.rf.6f9c22e73309596532ebea8f0fbd98f5.txt
✅ Обновлён: 163_png.rf.f7a5769efc58820e5756f8a202162a90.txt
✅ Обновлён: 4035_png.rf.67e3bacb5b05c3ac19f467bc93eb9f91.txt
ℹ️ Без изменений: 791_png.rf.e383461c92d05474d83ea0bb01f6b2d7.txt
✅ Обновлён: 142_png.rf.4410eba54c16424527a8409b65978f4f.txt
ℹ️ Без изменений: 1067_png.rf.77ef61815949b087288c61dd799d9457.txt
✅ Обновлён: 115_png.rf.415bc043008473783aab1655db980187.txt
ℹ️ Без изменений: 775_png.rf.10931b1a5f6adeaefe3d30e68d273a2c.txt
ℹ️ Без изменений: 765_png.rf.db49a8f253b20ad894af48348ae20acc.txt
✅ Обновлён: 60_png.rf.cf628977783bd88bdaa6799f63cfd6f9.txt
✅ Обновлён: 1044_png_jpg.rf.82298b0bc3d774b448daf5cb41359cf8.txt
✅ Обновлён: 4027_png.rf.9ad65b2464eab2563ef48ec5f7a39ad8.txt
✅ Обновлён: 877_png_jpg.rf.2f5f1fb63a66412824330052446f9c83.txt
✅ Обновлён: 4092_png.rf.6d2c7724bacff411deeea692ca08020b.txt
✅ Обновлё

Если остались файлы без нужной разметки, их можно удалить.

In [ ]:
# Пути к папкам относительно текущей директории запуска
labels_dir = "../data/full_door_window/train/labels"
images_dir = "../data/full_door_window/train/images"

In [29]:
# Пути к папкам относительно текущей директории запуска
labels_dir = "../data/full_door_window/valid/labels"
images_dir = "../data/full_door_window/valid/images"

In [27]:
# Пути к папкам относительно текущей директории запуска
labels_dir = "../data/full_door_window/test/labels"
images_dir = "../data/full_door_window/test/images"

In [30]:
# Поддерживаемые расширения изображений
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

# Список удалённых пар (label, image)
removed_files = []

# Проходим по всем файлам в папке labels
for label_file in os.listdir(labels_dir):
    if label_file.endswith(".txt"):
        label_path = os.path.join(labels_dir, label_file)
        
        # Если файл пустой
        if os.stat(label_path).st_size == 0:
            # Имя без расширения
            filename_wo_ext = os.path.splitext(label_file)[0]
            print('удаляем разметку:', filename_wo_ext)
            # Удаляем пустой txt
            os.remove(label_path)

            # Пытаемся найти и удалить соответствующее изображение
            image_deleted = False
            for ext in image_extensions:
                image_path = os.path.join(images_dir, filename_wo_ext + ext)
                if os.path.exists(image_path):
                    print('удаляем изображение:', filename_wo_ext)
                    os.remove(image_path)
                    removed_files.append((label_file, filename_wo_ext + ext))
                    image_deleted = True
print('всего удалено:', len(removed_files))
removed_files


удаляем разметку: 843_png.rf.4ff01eef7caa014e197818d24b606482
удаляем изображение: 843_png.rf.4ff01eef7caa014e197818d24b606482
удаляем разметку: 639_png.rf.337d7b715fcb9205a534d20bfc4b5799
удаляем изображение: 639_png.rf.337d7b715fcb9205a534d20bfc4b5799
удаляем разметку: 847_png.rf.b50635d91e5ed3b051c3ebc0d4e97693
удаляем изображение: 847_png.rf.b50635d91e5ed3b051c3ebc0d4e97693
удаляем разметку: 878_png.rf.c99255c450c6a228cbdd5988921e424f
удаляем изображение: 878_png.rf.c99255c450c6a228cbdd5988921e424f
удаляем разметку: 585_png.rf.94a309e6214ebb58bdd89c16adaa57a9
удаляем изображение: 585_png.rf.94a309e6214ebb58bdd89c16adaa57a9
удаляем разметку: 844_png.rf.93872d2197892657e47ea14a7dfec199
удаляем изображение: 844_png.rf.93872d2197892657e47ea14a7dfec199
удаляем разметку: 1009_png.rf.eaf790fa93055918642f09de81ab0155
удаляем изображение: 1009_png.rf.eaf790fa93055918642f09de81ab0155
удаляем разметку: 678_png.rf.03ebc6673a318d82d04e3aeedde89c23
удаляем изображение: 678_png.rf.03ebc6673a318d8

[('843_png.rf.4ff01eef7caa014e197818d24b606482.txt',
  '843_png.rf.4ff01eef7caa014e197818d24b606482.jpg'),
 ('639_png.rf.337d7b715fcb9205a534d20bfc4b5799.txt',
  '639_png.rf.337d7b715fcb9205a534d20bfc4b5799.jpg'),
 ('847_png.rf.b50635d91e5ed3b051c3ebc0d4e97693.txt',
  '847_png.rf.b50635d91e5ed3b051c3ebc0d4e97693.jpg'),
 ('878_png.rf.c99255c450c6a228cbdd5988921e424f.txt',
  '878_png.rf.c99255c450c6a228cbdd5988921e424f.jpg'),
 ('585_png.rf.94a309e6214ebb58bdd89c16adaa57a9.txt',
  '585_png.rf.94a309e6214ebb58bdd89c16adaa57a9.jpg'),
 ('844_png.rf.93872d2197892657e47ea14a7dfec199.txt',
  '844_png.rf.93872d2197892657e47ea14a7dfec199.jpg'),
 ('1009_png.rf.eaf790fa93055918642f09de81ab0155.txt',
  '1009_png.rf.eaf790fa93055918642f09de81ab0155.jpg'),
 ('678_png.rf.03ebc6673a318d82d04e3aeedde89c23.txt',
  '678_png.rf.03ebc6673a318d82d04e3aeedde89c23.jpg'),
 ('626_png.rf.e8b7d3b980230c1b558cd8d1932a5afc.txt',
  '626_png.rf.e8b7d3b980230c1b558cd8d1932a5afc.jpg'),
 ('855_png.rf.c544922475d0803147003

In [25]:
print('всего удалено:', len(removed_files))
removed_files

всего удалено: 161


[('1021_png.rf.c96c279df0c0742d770578633a161836.txt',
  '1021_png.rf.c96c279df0c0742d770578633a161836.jpg'),
 ('629_png.rf.19f7f7eca98738b1814b8212755d35e4.txt',
  '629_png.rf.19f7f7eca98738b1814b8212755d35e4.jpg'),
 ('763_png.rf.fe6b77725249807f0b1a0a16ef9ff21f.txt',
  '763_png.rf.fe6b77725249807f0b1a0a16ef9ff21f.jpg'),
 ('1065_png.rf.d43e28776e66107bd436cc39b850d157.txt',
  '1065_png.rf.d43e28776e66107bd436cc39b850d157.jpg'),
 ('946_png.rf.4db1fa57a4b78b797e73fe7541b800e9.txt',
  '946_png.rf.4db1fa57a4b78b797e73fe7541b800e9.jpg'),
 ('586_png.rf.b67b3bf6ee9c0abdfb369d0815c5a39c.txt',
  '586_png.rf.b67b3bf6ee9c0abdfb369d0815c5a39c.jpg'),
 ('865_png.rf.1715946f731840d4c68e93614e8225a0.txt',
  '865_png.rf.1715946f731840d4c68e93614e8225a0.jpg'),
 ('819_png.rf.8587f31aa49af8d84db94c9be1ceb11c.txt',
  '819_png.rf.8587f31aa49af8d84db94c9be1ceb11c.jpg'),
 ('964_png.rf.7ebd7b8bbceb147b1067fe404e298deb.txt',
  '964_png.rf.7ebd7b8bbceb147b1067fe404e298deb.jpg'),
 ('962_png.rf.b367d47445062b52c03